In [ ]:
import numpy as np 
import pandas as pd 


import os
for dirname, _, filenames in os.walk('\DSspecialization\Capstone_3'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
data = pd.read_csv("links.csv")
data = pd.read_csv("movies.csv")
data = pd.read_csv("ratings.csv")
data = pd.read_csv("tags.csv")
data.head()

In [ ]:
users = data['userId'].unique()
movies = data['movieId'].unique()

print("Number of users", len(users))
print("Number of movies", len(movies))

In [ ]:
test = pd.DataFrame(columns=data.columns)
train = pd.DataFrame(columns=data.columns)

test_ratio = 0.2

for u in users:
    temp = data[data['userId']==u]
    n = len(temp)
    test_size = int(test_ratio * n)
    
temp = temp.sort_values('timestamp').reset_index()
temp.drop('index', axis=1, inplace=True)

dummy_test = temp.loc[n-1-test_size:]
dummy_train = temp.loc[:n-2-test_size]

test = pd.concat([test, dummy_test])
train = pd.concat([train, dummy_train])

In [ ]:
test

In [ ]:
train[train['userId']==610].sort_values('timestamp').head()

In [ ]:
test[test['userId']==610].sort_values('timestamp').head()

In [ ]:
import numpy as np
def create_utility_matrix(data):
    
    movieField = data['movieId'].tolist()
    userField = data['userId'].tolist()
    ratingField = data['rating'].tolist()
    
    users = list(set(userField))
    movies = list(set(movieField))
    
    users_index = {users[i]: i for i in range(len(users))}
    
    pd_dict = {movie: [np.nan for i in range(len(users))] for movie in movies}
    
    for i in range(0, len(data)):
        movie = movieField[i]
        user = userField[i]
        rating = ratingField[i]
        pd_dict[movie][users_index[user]] = rating
    
    X = pd.DataFrame(pd_dict)
    X.index = users
    
    movies_cols = list(X.columns)
    movies_index = {movies_cols[i]: i for i in range(len(movies_cols))}
    
    return X, users_index, movies_index

In [ ]:
 users_index, movies_index = create_utility_matrix(data)

In [52]:
print(data)

      userId  movieId               tag   timestamp
0          2    60756             funny  1445714994
1          2    60756   Highly quotable  1445714996
2          2    60756      will ferrell  1445714992
3          2    89774      Boxing story  1445715207
4          2    89774               MMA  1445715200
...      ...      ...               ...         ...
3678     606     7382         for katie  1171234019
3679     606     7936           austere  1173392334
3680     610     3265            gun fu  1493843984
3681     610     3265  heroic bloodshed  1493843978
3682     610   168248  Heroic Bloodshed  1493844270

[3683 rows x 4 columns]


In [53]:
data[(data['userId']==609) & (data['movieId']==1)]

,userId,movieId,tag,timestamp


In [54]:

import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib.animation as animation

class MatFactor():
    #overiding the init function by adding the following parameters as arguments (the function self, Ratings, number of workers, learning rates/weights, #iterations)
    def __init__(self, R, K, alpha, beta, iterations):
        #assigning the values of variables 
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

        
        xs, ys = self.R.nonzero()
        self.z = [self.R[x, y] for x, y in zip(xs, ys)]
        self.x = [0 for i in range(len(self.z))]
        self.y = [i for i in range(len(self.z))]
        self.realLine = np.array([self.x, self.y, self.z])
        self.log = [self.realLine]
        self.shiftXIndex = 0

    def getShiftX(self):
        self.shiftXIndex += .5
        return [self.shiftXIndex for i in range(len(self.z))]

    def train(self):
        # Initialize user and item latent feature matrix
        self.P = np.random.normal(
            scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(
            scale=1./self.K, size=(self.num_items, self.K))

        # Initialize the biases
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        # general_mean of non zero values
        self.b = np.mean(self.R[np.where(self.R != 0)])

        # Create a list of training samples
        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]

        # Perform stochastic gradient descent for number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            print("Iteration: %d ; error = %.4f" % (i+1, mse))

        return training_process
    #defining function for mean square error
    def mse(self):
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        newPredictLine = np.array(
            [self.getShiftX(), self.y, [predicted[x, y] for x, y in zip(xs, ys)]])

        self.log.append(newPredictLine)
        error = 0
        
        #using for loop for calculating mean square error
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)
    
    #function for performing stochastic gradient descent
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_rating(i, j)
            e = (r - prediction)
            self.b_u[i] += self.alpha * (2*e - self.beta * self.b_u[i]) #PERFORM UPDATION
            self.b_i[j] += self.alpha * (2*e - self.beta * self.b_i[j])

            self.P[i, :] += self.alpha * \
                (e * self.Q[j, :] - self.beta * self.P[i, :])
            self.Q[j, :] += self.alpha * \
                (e * self.P[i, :] - self.beta * self.Q[j, :])

    def get_rating(self, i, j):
        prediction = self.b + self.b_u[i] + \
            self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    def full_matrix(self):
        return self.b + self.b_u[:, np.newaxis] + self.b_i[np.newaxis:, ] + self.P.dot(self.Q.T)


R = np.random.randint(6, size=(5, 5)) #GENERATE Ratings at random
mf = MatFactor(R, K=2, alpha=0.1, beta=0.01, iterations=100) #TRIAL-AND-ERROR FOR THESE HYPERPARAMETERS

mf.train()

Iteration: 1 ; error = 5.9933
Iteration: 2 ; error = 5.1236
Iteration: 3 ; error = 4.6849
Iteration: 4 ; error = 4.1315
Iteration: 5 ; error = 3.7317
Iteration: 6 ; error = 3.3801
Iteration: 7 ; error = 2.8200
Iteration: 8 ; error = 2.3710
Iteration: 9 ; error = 1.9616
Iteration: 10 ; error = 1.6634
Iteration: 11 ; error = 1.4120
Iteration: 12 ; error = 1.2371
Iteration: 13 ; error = 1.1762
Iteration: 14 ; error = 1.0648
Iteration: 15 ; error = 0.9654
Iteration: 16 ; error = 0.8843
Iteration: 17 ; error = 0.8305
Iteration: 18 ; error = 0.7772
Iteration: 19 ; error = 0.7332
Iteration: 20 ; error = 0.6732
Iteration: 21 ; error = 0.6080
Iteration: 22 ; error = 0.5979
Iteration: 23 ; error = 0.5043
Iteration: 24 ; error = 0.4913
Iteration: 25 ; error = 0.4710
Iteration: 26 ; error = 0.4201
Iteration: 27 ; error = 0.3734
Iteration: 28 ; error = 0.3428
Iteration: 29 ; error = 0.2981
Iteration: 30 ; error = 0.3093
Iteration: 31 ; error = 0.3363
Iteration: 32 ; error = 0.2907
Iteration: 33 ; e

[(0, 5.993287809230817),
 (1, 5.1236033910659184),
 (2, 4.684923051772883),
 (3, 4.131520242874223),
 (4, 3.7316890603342414),
 (5, 3.380070658618932),
 (6, 2.8199669302264114),
 (7, 2.3710318826666335),
 (8, 1.9616313180241374),
 (9, 1.6634315928103922),
 (10, 1.4120384916862474),
 (11, 1.2370977143624564),
 (12, 1.1762010173944537),
 (13, 1.0647922610000635),
 (14, 0.9653646892219908),
 (15, 0.8842992026084504),
 (16, 0.8304830035462927),
 (17, 0.777226286538293),
 (18, 0.7332121313791865),
 (19, 0.6732299723742046),
 (20, 0.6079670956717732),
 (21, 0.5978774723069371),
 (22, 0.5042623204100024),
 (23, 0.4913362373181585),
 (24, 0.47102988081631075),
 (25, 0.420050837972389),
 (26, 0.37342468594109024),
 (27, 0.34282528310924126),
 (28, 0.2981442245058258),
 (29, 0.3093371626162114),
 (30, 0.33626778163044085),
 (31, 0.290670665990905),
 (32, 0.2783399026105147),
 (33, 0.269467137350408),
 (34, 0.23251316057281635),
 (35, 0.23982345755834297),
 (36, 0.2171817821440705),
 (37, 0.20591

In [55]:
def movie_embedding_norm(models):
  """Visualizes the norm and number of ratings of the movie embeddings.
  Args:
    model: A MFModel object.
  """
  if not isinstance(models, list):
    models = [models]
  df = pd.DataFrame({
      'title': movies['title'],
      'genre': movies['genre'],
      'num_ratings': movies_ratings['rating count'],
  })
  charts = []
  brush = alt.selection_interval()
  for i, model in enumerate(models):
    norm_key = 'norm'+str(i)
    df[norm_key] = np.linalg.norm(model.embeddings["movie_id"], axis=1)
    nearest = alt.selection(
        type='single', encodings=['x', 'y'], on='mouseover', nearest=True,
        empty='none')
    base = alt.Chart().mark_circle().encode(
        x='num_ratings',
        y=norm_key,
        color=alt.condition(brush, alt.value('#4c78a8'), alt.value('lightgray'))
    ).properties(
        selection=nearest).add_selection(brush)
    text = alt.Chart().mark_text(align='center', dx=5, dy=-5).encode(
        x='num_ratings', y=norm_key,
        text=alt.condition(nearest, 'title', alt.value('')))
    charts.append(alt.layer(base, text))
  return alt.hconcat(*charts, data=df)

def visualize_movie_embeddings(data, x, y):
  nearest = alt.selection(
      type='single', encodings=['x', 'y'], on='mouseover', nearest=True,
      empty='none')
  base = alt.Chart().mark_circle().encode(
      x=x,
      y=y,
      color=alt.condition(genre_filter, "genre", alt.value("whitesmoke")),
  ).properties(
      width=600,
      height=600,
      selection=nearest)
  text = alt.Chart().mark_text(align='left', dx=5, dy=-5).encode(
      x=x,
      y=y,
      text=alt.condition(nearest, 'title', alt.value('')))
  return alt.hconcat(alt.layer(base, text), genre_chart, data=data)

def tsne_movie_embeddings(model):
  """Visualizes the movie embeddings, projected using t-SNE with Cosine measure.
  Args:
    model: A MFModel object.
  """
  tsne = sklearn.manifold.TSNE(
      n_components=2, perplexity=40, metric='cosine', early_exaggeration=10.0,
      init='pca', verbose=True, n_iter=400)

  print('Running t-SNE...')
  V_proj = tsne.fit_transform(model.embeddings["movie_id"])
  movies.loc[:,'x'] = V_proj[:, 0]
  movies.loc[:,'y'] = V_proj[:, 1]
  return visualize_movie_embeddings(movies, 'x', 'y')
     


     

In [56]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
print(movie_embedding_norm)
     

<function movie_embedding_norm at 0x0000025FF253BAF0>


In [ ]:
model_lowinit = build_model(ratings, embedding_dim=30, init_stddev=0.05)
model_lowinit.train(num_iterations=1000, learning_rate=10.)
movie_neighbors(model_lowinit, "Aladdin", DOT)
movie_neighbors(model_lowinit, "Aladdin", COSINE)
movie_embedding_norm([model, model_lowinit])

In [58]:
print(tsne_movie_embeddings)

<function tsne_movie_embeddings at 0x0000025FE963A8B0>
